In [1]:

import sys  
sys.path.insert(0, r"C:\\Users\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\druida_V01\\src\\")

import os

from __future__ import print_function
#from Utilities.SaveAnimation import Video



from druida import Stack
from druida import setup

from druida.DataManager import datamanager
from druidaHFSS.modules import tools
from druida.tools import utils

import random
import numpy as np
import pandas as pd
from numpy.linalg import norm
from scipy.signal import find_peaks
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAccuracy




import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.offsetbox import OffsetImage,AnchoredOffsetbox
import matplotlib.image as image

from IPython.display import HTML

from glob import glob

import argparse





In [2]:
parser = argparse.ArgumentParser()

parser.add_argument("run_name",type=str)
parser.add_argument("epochs",type=int)
parser.add_argument("batch_size",type=int)
parser.add_argument("workers",type=int)
parser.add_argument("gpu_number",type=int)
parser.add_argument("device",type=str)
parser.add_argument("learning_rate",type=float)
parser.add_argument("condition_len",type=float) #This defines the length of our conditioning vector
parser.add_argument("metricType",type=float) #This defines the length of our conditioning vector

parser.run_name = "DDPM_Uncondtional"
parser.epochs = 6
parser.batch_size = 10
parser.workers=0
parser.gpu_number=0
parser.image_size = 512
parser.dataset_path = os.path.normpath('/content/drive/MyDrive/Training_Data/Training_lite/')
parser.device = "cpu"
parser.learning_rate = 1e-6
parser.condition_len = 10
parser.metricType='AbsorbanceTM' #this is to be modified when training for different metrics.

categories=["box", "circle"]

In [ ]:
PATH_TM = './trainedModelTM_abs_Multitarget_cross.pth'
PATH_TE = './trainedModelTE_abs_Multitarget.pth'

boxImagesPath="C:\\Users\\jorge\\Dropbox\\Public\\MetasufacesData\\testImages\\"
DataPath="C:\\Users\\jorge\\Dropbox\\Public\\MetasufacesData\\Exports\\output\\"
simulationData="C:\\Users\\jorge\\Dropbox\\Public\\MetasufacesData\\DBfiles\\"